In [1]:
library(tidyverse)
library(RPostgreSQL)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 2.2.1     ✔ purrr   0.2.4
✔ tibble  1.3.4     ✔ dplyr   0.7.4
✔ tidyr   0.7.2     ✔ stringr 1.2.0
✔ readr   1.1.1     ✔ forcats 0.2.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required package: DBI


In [2]:
source("utils.R")

In [3]:
sql_dir <- file.path("..", "sql")
list.files(sql_dir)

[1] "basics.sql"              "cohort.sql"             
[3] "icd9.sql"                "lab_tests.sql"          
[5] "lab_unpivot.sql"         "merge_data.sql"         
[7] "population.sql"          "vital_signs.sql"        
[9] "vital_signs_unpivot.sql"

In [4]:
drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname = "mimic")
dbSendQuery(con, "set search_path=echo,public,mimiciii;")

<PostgreSQLResult>

In [5]:
sql <- readr::read_file(file.path(sql_dir, "merge_data.sql")) %>%
    make_view_sql("merged_data", "materialized view")

In [6]:
dbSendQuery(con, sql)

<PostgreSQLResult>

In [7]:
sql <- "
select * from merged_data limit 10;
"
dbGetQuery(con, sql)

hadm_id,icustay_id,subject_id,first_careunit,intime,outtime,angus,age,icu_order,echo_time,⋯,lab_sodium_min,lab_sodium_max,lab_chloride_flag,lab_chloride_first,lab_chloride_min,lab_chloride_max,lab_ph_flag,lab_ph_first,lab_ph_min,lab_ph_max
125078,201220,66690,MICU,2106-04-27 01:47:50,2106-05-01 11:25:46,1,62.67646,1,NA,⋯,160,165,1,123,123,128,1,7.45,7.42,7.45
151232,215842,11663,MICU,2188-02-14 01:48:15,2188-02-15 19:02:48,1,86.76186,1,NA,⋯,139,139,1,105,105,105,0,NA,NA,NA
164444,234312,86645,SICU,2165-06-22 01:47:16,2165-07-07 13:55:20,1,56.08904,1,NA,⋯,142,146,1,108,107,112,1,7.49,7.49,7.49
146726,289157,10304,MICU,2156-06-23 14:26:00,2156-06-30 09:26:00,1,45.91093,1,2156-06-25,⋯,134,136,1,100,100,105,1,7.44,7.44,7.44
160170,211964,94534,MICU,2160-03-05 14:23:19,2160-03-06 22:48:41,1,59.38693,1,NA,⋯,138,138,1,103,103,103,0,NA,NA,NA
112553,230173,31544,MICU,2140-01-30 20:39:25,2140-02-02 18:41:39,1,300.00345,1,NA,⋯,143,147,1,118,113,118,0,NA,NA,NA
151997,210817,28611,MICU,2122-09-24 18:50:10,2122-09-28 22:08:48,1,66.40196,1,NA,⋯,131,148,1,97,97,120,1,7.61,7.43,7.61
158611,242287,6485,MICU,2185-06-09 19:12:12,2185-06-10 18:33:31,1,83.10326,1,NA,⋯,129,129,1,98,98,98,0,NA,NA,NA
105158,284489,1987,SICU,2102-03-31 03:46:10,2102-04-03 13:05:57,1,45.42785,1,2102-03-31,⋯,146,151,1,108,104,108,1,6.82,6.82,7.30
194870,251858,95933,MICU,2146-07-17 12:44:06,2146-07-19 11:45:06,1,76.12085,1,2146-07-18,⋯,135,136,1,99,99,104,1,7.09,7.09,7.33


In [8]:
dbDisconnect(con)
dbUnloadDriver(drv)

[1] TRUE

[1] TRUE